# Introduction
Determin Porygon DT of Sampled Users to Each Restaurant


# Data collection
We get the data from big query.

In [0]:
# load packages
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.io import gbq
from google.cloud import bigquery
from google.colab import drive # to allow colab save file in my drive
drive.mount('drive')
cwd=os.getcwd()
cwd

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


'/content'

In [0]:
query="""
select * from `fulfillment-dwh-production.temp.results_tw_7`
"""
# run the query and save the results as data frame named df
df=gbq.read_gbq(query,project_id="fulfillment-dwh-production")


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=725825577420-unm2gnkiprugilg743tkbig250f4sfsj.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=aZdtW1LMzuZEVt37iNjhRr8yYlBFQb&prompt=consent&access_type=offline
Enter the authorization code: 4/ywFsu6tbRhP-w_bhSMwKk_7I9jlQpG3BsWZhLPnJxp2-jMthPYtYZs8


In [0]:
# to save money, next time we can read the csv file instead of query from bigquery
#df.to_csv("taiwan_geohash7.csv", sep="\t")
#!cp taiwan_geohash7.csv "drive/My Drive/dataset"
#df = pd.read_csv("/content/taiwan_geohash7.csv")
df.head()

,user_zone_id,geohash,geohash_center,restaurant_id,weight,porygon_dt
0,3,wsqq7fh,POINT(121.460037231445 25.0165557861328),y6oa,79.0,14
1,5,wsqqsr5,POINT(121.48063659668 25.0879669189453),e7zo,78.0,7
2,3,wsqq5zd,POINT(121.457290649414 25.0028228759766),e6mx,78.0,6
3,6,wsqqm7q,POINT(121.528701782227 25.0234222412109),e1qz,92.0,12
4,5,wsqqsej,POINT(121.494369506836 25.0659942626953),y4yq,77.0,4


In [0]:
pivot=df.pivot_table("restaurant_id", index=['geohash_center',"geohash","weight"], columns='porygon_dt', aggfunc='count')
totaldistinct=len(pivot) # total distinct geohashes
pivot.head()

,,porygon_dt,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
geohash_center,geohash,weight,,,,,,,,,,,,,,,,,,,
POINT(121.343307495117 25.0440216064453),wsqq3pp,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,4.0,4.0,10.0,16.0,31.0,34.0,40.0,25.0,24.0,14.0
POINT(121.343307495117 25.0838470458984),wsqq9nr,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3.0,2.0,4.0,11.0,23.0,39.0,25.0,8.0,11.0,26.0,43.0
POINT(121.343307495117 25.0865936279297),wsqq9nz,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,NaN,5.0,15.0,25.0,35.0,23.0,7.0,12.0,26.0
POINT(121.344680786133 25.0399017333984),wsqq3q2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,7.0,14.0,6.0,31.0,42.0,39.0,43.0,5.0,7.0
POINT(121.344680786133 25.0412750244141),wsqq3q8,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,2.0,7.0,14.0,7.0,35.0,41.0,42.0,30.0,9.0,15.0


In [0]:
pivot_cumsum = pivot.cumsum(axis=1)
pivot_cumsum = pivot_cumsum.fillna(0) # replace NA with 0
pivot_cumsum.head()

,,porygon_dt,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
geohash_center,geohash,weight,,,,,,,,,,,,,,,,,,,
POINT(121.343307495117 25.0440216064453),wsqq3pp,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,6.0,10.0,20.0,36.0,67.0,101.0,141.0,166.0,190.0,204.0
POINT(121.343307495117 25.0838470458984),wsqq9nr,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,5.0,7.0,11.0,22.0,45.0,84.0,109.0,117.0,128.0,154.0,197.0
POINT(121.343307495117 25.0865936279297),wsqq9nz,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,6.0,0.0,11.0,26.0,51.0,86.0,109.0,116.0,128.0,154.0
POINT(121.344680786133 25.0399017333984),wsqq3q2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,9.0,23.0,29.0,60.0,102.0,141.0,184.0,189.0,196.0
POINT(121.344680786133 25.0412750244141),wsqq3q8,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,4.0,11.0,25.0,32.0,67.0,108.0,150.0,180.0,189.0,204.0


# Weighted version

In [0]:
weight = pivot_cumsum
weight = weight.reset_index()
weight

porygon_dt,geohash_center,geohash,weight,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,POINT(121.343307495117 25.0440216064453),wsqq3pp,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,6.0,10.0,20.0,36.0,67.0,101.0,141.0,166.0,190.0,204.0
1,POINT(121.343307495117 25.0838470458984),wsqq9nr,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,5.0,7.0,11.0,22.0,45.0,84.0,109.0,117.0,128.0,154.0,197.0
2,POINT(121.343307495117 25.0865936279297),wsqq9nz,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,6.0,0.0,11.0,26.0,51.0,86.0,109.0,116.0,128.0,154.0
3,POINT(121.344680786133 25.0399017333984),wsqq3q2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,9.0,23.0,29.0,60.0,102.0,141.0,184.0,189.0,196.0
4,POINT(121.344680786133 25.0412750244141),wsqq3q8,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,4.0,11.0,25.0,32.0,67.0,108.0,150.0,180.0,189.0,204.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14118,POINT(121.674270629883 25.0824737548828),wsqw8y0,6.0,0.0,0.0,2.0,0.0,4.0,0.0,10.0,27.0,30.0,48.0,66.0,82.0,107.0,115.0,127.0,131.0,138.0,141.0,148.0
14119,POINT(121.675643920898 25.0797271728516),wsqw8v9,10.0,0.0,0.0,1.0,0.0,2.0,4.0,0.0,10.0,24.0,33.0,51.0,68.0,84.0,108.0,119.0,126.0,132.0,139.0,141.0
14120,POINT(121.675643920898 25.0811004638672),wsqw8vc,16.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,13.0,28.0,36.0,56.0,71.0,86.0,110.0,121.0,129.0,132.0,140.0,143.0
14121,POINT(121.675643920898 25.0838470458984),wsqw8y3,2.0,0.0,0.0,1.0,2.0,3.0,0.0,4.0,15.0,24.0,37.0,56.0,68.0,91.0,110.0,119.0,129.0,134.0,139.0,142.0


In [0]:
numOfRestaurants = [1, 5, 10, 15, 20, 25, 30]
drivingTimesinMin = range(2,21)
resArray = []
for indexNumOfRestaurants in numOfRestaurants:
    sumByDrivingTimes = []
    for indexDrivingTimesInMin in drivingTimesinMin:
        sumByDrivingTimes.append(weight.loc[weight[indexDrivingTimesInMin] >= indexNumOfRestaurants, "weight"].sum())
        
    sumByDrivingTimes.insert(0,indexNumOfRestaurants)
    resArray.append(sumByDrivingTimes)

resArray

[[1,
  283795.0,
  323575.0,
  335912.0,
  341094.0,
  343542.0,
  345161.0,
  345893.0,
  346892.0,
  347362.0,
  347901.0,
  347548.0,
  347012.0,
  346640.0,
  346695.0,
  345051.0,
  342467.0,
  334634.0,
  326331.0,
  317919.0],
 [5,
  172925.0,
  287148.0,
  320256.0,
  333205.0,
  339458.0,
  342705.0,
  344336.0,
  345837.0,
  346856.0,
  347632.0,
  347414.0,
  346960.0,
  346612.0,
  346675.0,
  345048.0,
  342460.0,
  334630.0,
  326330.0,
  317917.0],
 [10,
  98614.0,
  250095.0,
  304002.0,
  324244.0,
  334188.0,
  339909.0,
  342517.0,
  344311.0,
  345940.0,
  346974.0,
  347129.0,
  346797.0,
  346517.0,
  346628.0,
  345015.0,
  342442.0,
  334625.0,
  326325.0,
  317910.0],
 [15,
  54576.0,
  212324.0,
  287144.0,
  315765.0,
  329563.0,
  336791.0,
  340832.0,
  343053.0,
  344751.0,
  346198.0,
  346556.0,
  346429.0,
  346299.0,
  346523.0,
  344944.0,
  342407.0,
  334607.0,
  326320.0,
  317909.0],
 [20,
  31444.0,
  177331.0,
  269351.0,
  307130.0,
  324495.0,

In [0]:
colNames = np.append("Number of Restaurants", drivingTimesinMin)

df2 = pd.DataFrame(resArray,
                   columns = colNames)
df2=df2.set_index('Number of Restaurants')
df2

,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Number of Restaurants,,,,,,,,,,,,,,,,,,,
1,283795.0,323575.0,335912.0,341094.0,343542.0,345161.0,345893.0,346892.0,347362.0,347901.0,347548.0,347012.0,346640.0,346695.0,345051.0,342467.0,334634.0,326331.0,317919.0
5,172925.0,287148.0,320256.0,333205.0,339458.0,342705.0,344336.0,345837.0,346856.0,347632.0,347414.0,346960.0,346612.0,346675.0,345048.0,342460.0,334630.0,326330.0,317917.0
10,98614.0,250095.0,304002.0,324244.0,334188.0,339909.0,342517.0,344311.0,345940.0,346974.0,347129.0,346797.0,346517.0,346628.0,345015.0,342442.0,334625.0,326325.0,317910.0
15,54576.0,212324.0,287144.0,315765.0,329563.0,336791.0,340832.0,343053.0,344751.0,346198.0,346556.0,346429.0,346299.0,346523.0,344944.0,342407.0,334607.0,326320.0,317909.0
20,31444.0,177331.0,269351.0,307130.0,324495.0,334273.0,338993.0,342041.0,343791.0,345368.0,346003.0,345775.0,345942.0,346231.0,344823.0,342368.0,334549.0,326283.0,317899.0
25,17716.0,148490.0,250762.0,296481.0,317431.0,329679.0,336870.0,340823.0,343043.0,344718.0,345579.0,345319.0,345581.0,345949.0,344723.0,342235.0,334511.0,326253.0,317887.0
30,9783.0,122788.0,234438.0,286240.0,311534.0,324393.0,332741.0,338380.0,341895.0,344027.0,344863.0,345109.0,345256.0,345623.0,344385.0,342049.0,334383.0,326218.0,317856.0


In [0]:
totalnum = weight["weight"].sum()
df2[["2","3","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20"]].div(totalnum)

,2,3,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Number of Restaurants,,,,,,,,,,,,,,,,,,,,
1,0.812068,0.925897,0.925897,0.961199,0.976027,0.983032,0.987664,0.989759,0.992617,0.993962,0.995505,0.994495,0.992961,0.991896,0.992054,0.987349,0.979955,0.957542,0.933783,0.909712
5,0.494818,0.821662,0.821662,0.916400,0.953453,0.971345,0.980637,0.985304,0.989599,0.992514,0.994735,0.994111,0.992812,0.991816,0.991996,0.987341,0.979935,0.957530,0.933780,0.909707
10,0.282180,0.715637,0.715637,0.869889,0.927811,0.956265,0.972636,0.980099,0.985232,0.989893,0.992852,0.993296,0.992346,0.991544,0.991862,0.987246,0.979884,0.957516,0.933766,0.909687
15,0.156167,0.607557,0.607557,0.821651,0.903549,0.943031,0.963714,0.975277,0.981632,0.986491,0.990632,0.991656,0.991293,0.990921,0.991562,0.987043,0.979784,0.957464,0.933751,0.909684
20,0.089976,0.507425,0.507425,0.770737,0.878840,0.928529,0.956509,0.970015,0.978736,0.983744,0.988257,0.990074,0.989421,0.989899,0.990726,0.986697,0.979672,0.957298,0.933646,0.909655
25,0.050694,0.424898,0.424898,0.717545,0.848368,0.908316,0.943363,0.963940,0.975251,0.981604,0.986397,0.988860,0.988116,0.988866,0.989919,0.986411,0.979292,0.957190,0.933560,0.909621
30,0.027994,0.351353,0.351353,0.670835,0.819064,0.891442,0.928237,0.952125,0.968261,0.978319,0.984419,0.986812,0.987515,0.987936,0.988986,0.985444,0.978759,0.956823,0.933460,0.909532


## Non-weighted version
Here we use another way to replicate the table.


In [0]:
result = pivot_cumsum.apply(pd.value_counts).fillna(0)
result["index"]=result.index
result

porygon_dt,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,index
0.0,6279.0,3467.0,2184.0,1485.0,1052.0,805.0,587.0,404.0,334.0,254.0,220.0,197.0,230.0,182.0,232.0,303.0,528.0,756.0,1084.0,0.0
1.0,1623.0,926.0,464.0,299.0,171.0,78.0,59.0,34.0,25.0,6.0,10.0,5.0,5.0,1.0,1.0,2.0,0.0,0.0,2.0,1.0
2.0,994.0,731.0,465.0,261.0,188.0,131.0,80.0,58.0,28.0,21.0,5.0,5.0,2.0,3.0,0.0,2.0,1.0,0.0,0.0,2.0
3.0,782.0,551.0,417.0,288.0,176.0,129.0,96.0,74.0,41.0,21.0,16.0,2.0,3.0,0.0,1.0,1.0,2.0,1.0,0.0,3.0
4.0,607.0,437.0,319.0,203.0,164.0,112.0,88.0,63.0,33.0,32.0,34.0,15.0,5.0,5.0,1.0,1.0,0.0,0.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3529.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3529.0
3534.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3534.0
3542.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3542.0
3544.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3544.0


In [0]:
column1=result[result["index"]>=1].sum()
column5=result[result["index"]>=5].sum()
column10=result[result["index"]>=10].sum()
column15=result[result["index"]>=15].sum()
column20=result[result["index"]>=20].sum()
column25=result[result["index"]>=25].sum()
column30=result[result["index"]>=30].sum()
aggregate=pd.concat([column1, column5, column10, column15, column20, column25, column30], axis=1)
aggregate.columns = [1,5,10,15,20,25,30]
aggregate=aggregate.T

In [0]:
aggregate[drivingTimesinMin].div(totaldistinct)

porygon_dt,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
1,0.555406,0.754514,0.845359,0.894852,0.925512,0.943001,0.958437,0.971394,0.976351,0.982015,0.984423,0.986051,0.983715,0.987113,0.983573,0.978546,0.962614,0.946470,0.923246
5,0.271755,0.567231,0.727466,0.820435,0.876018,0.911138,0.935566,0.955179,0.967358,0.976351,0.979820,0.984139,0.982652,0.986476,0.983360,0.978121,0.962402,0.946399,0.923104
10,0.141542,0.450188,0.643489,0.758691,0.832826,0.883382,0.913899,0.935566,0.953551,0.967925,0.974226,0.980599,0.980103,0.985131,0.982582,0.977484,0.962048,0.946116,0.922679
15,0.074488,0.357714,0.575090,0.712030,0.797706,0.855484,0.896339,0.920626,0.939885,0.958295,0.967287,0.975147,0.976138,0.982582,0.981165,0.976775,0.961340,0.945833,0.922609
20,0.041209,0.284076,0.512993,0.666360,0.767259,0.832897,0.880833,0.909934,0.928769,0.949090,0.959782,0.968279,0.971748,0.978900,0.979041,0.975784,0.960136,0.944771,0.922184
25,0.022658,0.229979,0.459180,0.623026,0.726899,0.805353,0.861786,0.897118,0.920059,0.941160,0.953551,0.963039,0.968066,0.976209,0.977342,0.973660,0.959145,0.944275,0.921759
30,0.012108,0.183672,0.413510,0.582950,0.697798,0.776747,0.835871,0.879700,0.909297,0.933159,0.947037,0.958366,0.963251,0.972244,0.974368,0.971677,0.957445,0.943213,0.921263


## Kepler Visualisation


In [0]:
# %%capture help hide the installing process
%%capture   
!pip install keplergl

In [0]:
from keplergl import KeplerGl 

In [0]:
kepler = pivot.reset_index()
kepler = kepler.fillna(0)
kepler

porygon_dt,geohash_center,geohash,weight,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,POINT(121.343307495117 25.0440216064453),wsqq3pp,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,4.0,4.0,10.0,16.0,31.0,34.0,40.0,25.0,24.0,14.0
1,POINT(121.343307495117 25.0838470458984),wsqq9nr,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,2.0,4.0,11.0,23.0,39.0,25.0,8.0,11.0,26.0,43.0
2,POINT(121.343307495117 25.0865936279297),wsqq9nz,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,5.0,15.0,25.0,35.0,23.0,7.0,12.0,26.0
3,POINT(121.344680786133 25.0399017333984),wsqq3q2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,7.0,14.0,6.0,31.0,42.0,39.0,43.0,5.0,7.0
4,POINT(121.344680786133 25.0412750244141),wsqq3q8,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,7.0,14.0,7.0,35.0,41.0,42.0,30.0,9.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14118,POINT(121.674270629883 25.0824737548828),wsqw8y0,6.0,0.0,0.0,2.0,0.0,2.0,0.0,6.0,17.0,3.0,18.0,18.0,16.0,25.0,8.0,12.0,4.0,7.0,3.0,7.0
14119,POINT(121.675643920898 25.0797271728516),wsqw8v9,10.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,6.0,14.0,9.0,18.0,17.0,16.0,24.0,11.0,7.0,6.0,7.0,2.0
14120,POINT(121.675643920898 25.0811004638672),wsqw8vc,16.0,0.0,0.0,1.0,0.0,2.0,1.0,0.0,9.0,15.0,8.0,20.0,15.0,15.0,24.0,11.0,8.0,3.0,8.0,3.0
14121,POINT(121.675643920898 25.0838470458984),wsqw8y3,2.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,11.0,9.0,13.0,19.0,12.0,23.0,19.0,9.0,10.0,5.0,5.0,3.0


In [0]:
col_list = list(kepler)
del col_list[0:3]
col_list

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [0]:
kepler["total_restaurant"]=kepler[col_list].sum(axis=1)
kepler

porygon_dt,geohash_center,geohash,weight,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,total_restaurant
0,POINT(121.343307495117 25.0440216064453),wsqq3pp,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,4.0,4.0,10.0,16.0,31.0,34.0,40.0,25.0,24.0,14.0,204.0
1,POINT(121.343307495117 25.0838470458984),wsqq9nr,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,2.0,4.0,11.0,23.0,39.0,25.0,8.0,11.0,26.0,43.0,197.0
2,POINT(121.343307495117 25.0865936279297),wsqq9nz,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,5.0,15.0,25.0,35.0,23.0,7.0,12.0,26.0,154.0
3,POINT(121.344680786133 25.0399017333984),wsqq3q2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,7.0,14.0,6.0,31.0,42.0,39.0,43.0,5.0,7.0,196.0
4,POINT(121.344680786133 25.0412750244141),wsqq3q8,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,7.0,14.0,7.0,35.0,41.0,42.0,30.0,9.0,15.0,204.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14118,POINT(121.674270629883 25.0824737548828),wsqw8y0,6.0,0.0,0.0,2.0,0.0,2.0,0.0,6.0,17.0,3.0,18.0,18.0,16.0,25.0,8.0,12.0,4.0,7.0,3.0,7.0,148.0
14119,POINT(121.675643920898 25.0797271728516),wsqw8v9,10.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,6.0,14.0,9.0,18.0,17.0,16.0,24.0,11.0,7.0,6.0,7.0,2.0,141.0
14120,POINT(121.675643920898 25.0811004638672),wsqw8vc,16.0,0.0,0.0,1.0,0.0,2.0,1.0,0.0,9.0,15.0,8.0,20.0,15.0,15.0,24.0,11.0,8.0,3.0,8.0,3.0,143.0
14121,POINT(121.675643920898 25.0838470458984),wsqw8y3,2.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,11.0,9.0,13.0,19.0,12.0,23.0,19.0,9.0,10.0,5.0,5.0,3.0,142.0


In [3]:
!pip install pydeck

In [0]:
import pydeck

In [10]:
map_1 = KeplerGl()
#map_1.add_data(data=kepler, name='data_1')
!pip install ipython

import IPython

IPython.display.HTML(map_1)

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


TypeError: ignored

In [7]:
map_1

KeplerGl()